# Environment & Config Check

In [5]:
import os, pathlib, datetime as dt
import requests
import pandas as pd
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from urllib.parse import urlparse
import time
from datetime import date
from dateutil.relativedelta import relativedelta
import re
import io
import zipfile
import os


import sys 
sys.path.append("..")
from src.config import load_env, get_key
load_env()


.env loaded (if present)


# Public Variable

In [6]:
api_key = get_key("API_KEY"); print(f"API_KEY present: {api_key is not None}")
import sys 
sys.path.append("..")
RAW = pathlib.Path("..") / "data/raw"; RAW.mkdir(parents=True, exist_ok=True)

API_KEY present: True


# Helper Method

In [7]:
def ts():
    return dt.datetime.now().strftime('%Y%m%d-%H%M%S')

def save_csv(df: pd.DataFrame, prefix: str, **meta):
    mid = '_'.join([f"{k}-{v}" for k,v in meta.items()])
    path = RAW / f"{prefix}_{mid}_{ts()}.csv"
    df.to_csv(path, index=False)
    print('Saved', path)
    return path

def validate(df: pd.DataFrame, required):
    missing = [c for c in required if c not in df.columns]
    return {'missing': missing, 'shape': df.shape, 'na_total': int(df.isna().sum().sum())}

# Stage2 - Data Acquisition - API

##     - Data in Real-time Availability - Need 10 mintues level update

In [8]:
#Citi Bike Station Status - real-time availability
url = "https://gbfs.citibikenyc.com/gbfs/en/station_status.json"
resp = requests.get(url)
js = resp.json()

In [9]:
data = js["data"]["stations"]
df_api = pd.DataFrame(data)
df_api = df_api.reset_index()
v_api = validate(df_api, ['date','close']); v_api
_ = save_csv(df_api, prefix='api', source='alpha' , symbol="Station Status")

Saved ../data/raw/api_source-alpha_symbol-Station Status_20250820-202642.csv


### ---Data Structure
{"data":{"Station":[{Station1},{Station2},{Station3}]},""last_updated": 1755700740, "ttl": 60, "version": "1.1"}

### ---Inside {Station1}
	•	station_id: 
	•	legacy_id: 
            # ID used in early stage to merge with old system
	•	is_installed:  
            # Is station physically installed? 1: yes 0: No
	•	is_renting: 
            # Is renting Allowed?
	•	is_returning: 
            # Is returning Allowed?
	•	num_bikes_available: 
	•	num_ebikes_available: 
            # How many avaliable electric bike
	•	num_bikes_disabled: 
            # How many bike not operating
	•	num_docks_available: 
	•	num_docks_disabled: 
            # How many docks not operating
	•	eightd_has_available_keys: 
            # keys for VIP avaliable?
	•	last_reported: 1754914801
            # use pd.to_datetime(..., unit="s") to convert to readable time

## Documentation
- API Source: 
    https://gbfs.citibikenyc.com/gbfs/en/station_status.json 
    •	Key parameters:
	    •symbol=AAPL
	    •outputsize=compact
	    •apikey=${ALPHAVANTAGE_API_KEY}（from .env）
- Scrape Source: https://en.wikipedia.org/wiki/List_of_S%26P_500_companies
- Assumptions & risks: 
    1:API rate limit: Alpha Vantage allow free user 5 request per mintue and 500 per day, frequent access would be banned from this API
    2:For parsing, the wikipidia's table structure may change, which may break parsing
    3: schema changes, which is backed up by the validation funciton
    4: Assuming the data source only has tolarable data completeness and consistency
- Confirm `.env` is not committed.

## Historical Rider Data
#    -data is tremendously big, I will only use the last three month data

In [11]:
# ===== 路径设置：把输出放到 notebook 的上一级目录中的另一个文件夹 =====
NOTEBOOK_DIR = pathlib.Path.cwd()      
PARENT_DIR = NOTEBOOK_DIR.parent            # 母文件夹
TARGET_DIR = PARENT_DIR / "data"/"raw"  # ← 你想要的“另一个文件夹”名称，改成你喜欢的
EXTRACT_DIR = TARGET_DIR / "extracted"        # 子文件夹：单表导出
OUTPUT_CSV = RAW / f"citibike_all_trips{ts()}.csv"

# ========== 你可以在这里设定过滤规则 ==========
YEARS = None          # 例：{2022, 2023, 2024}；为 None 表示不过滤
MAX_ZIPS = None       # 例：先测试只跑 5 个；为 None 表示不限
EXTRACT_DIR = "extracted"
TIMEOUT = 60
RETRIES = 3
LAST_N_MONTHS = 3           # 要抓取的最近 N 个月
INCLUDE_CURRENT_MONTH = False  # 是否把“当月”也算进去
INDEX_URL = "https://s3.amazonaws.com/tripdata"
# ============================================

def target_yearmonths(n=LAST_N_MONTHS, include_current=INCLUDE_CURRENT_MONTH):
    """返回形如 {'202507','202506','202505'} 的集合"""
    today = date.today()
    # 基准月：含当月则从当月开始；否则从上个月开始
    start_month = today if include_current else (today.replace(day=1) - relativedelta(days=1))
    cur = start_month.replace(day=1)  # 取每月 1 号方便回溯整月
    yms = []
    for _ in range(n):
        yms.append(f"{cur.year}{cur.month:02d}")
        cur = (cur.replace(day=1) - relativedelta(days=1)).replace(day=1)
    return set(yms)

def list_zip_urls(index_url=INDEX_URL):
    resp = requests.get(index_url, timeout=TIMEOUT)
    resp.raise_for_status()
    text = resp.text.strip()
    hrefs = []

    soup = BeautifulSoup(text, "xml")
    for loc in soup.find_all("Key"):
        key = loc.get_text().strip()
        if key.lower().endswith(".zip"):
            hrefs.append(urljoin(index_url + "/", key))

    hrefs = sorted(set(hrefs))

    # ===== 新增：仅保留形如 202411-citibike-tripdata.zip 的“月包”，并过滤到最近 N 个月 =====
    ym_set = target_yearmonths()
    month_zip_pattern = re.compile(r'/(\d{6})-citibike-tripdata\.zip$', re.IGNORECASE) #learned from GPT that this way, I can get the 6 digit code for month

    filtered = []
    for u in hrefs:
        # See if the URL matches with start with 6 digit time code with "-citibike-tripdata.zip"
        m = month_zip_pattern.search(u)
        if not m:
            continue  # 排除形如 2013-citibike-tripdata.zip 的“年包”或其他命名
        ym = m.group(1) #提取第一个捕获组，which is the 6 digit code
        if ym in ym_set:
            filtered.append(u)

    return filtered

# Goal of this function is to download the file from one URL write it to a buffer
def robust_get(url, timeout=TIMEOUT, retries=RETRIES, backoff=2.0):
    for i in range(retries):
        try:
            with requests.get(url, stream=True, timeout=timeout) as r:
                r.raise_for_status() #learned from GPT that this is a good way to handle errors
                                     # When we get 404 not found, it will raise an HTTPError, if not this code, then it will not
                                     # Some expected common status include:	  •	200 = success
                                                                            # •	404 = Not Found（找不到文件）
                                                                            # •	403 = Forbidden（权限问题）
                                                                            # •	500 = Internal Server Error（服务器错误）
                buf = io.BytesIO() #learned from GPT that this is a good way to handle large files(create buffer)
                for chunk in r.iter_content(chunk_size=1024 * 1024): #iterate 1MB by 1MB
                    if chunk:
                        buf.write(chunk) #Write it in to the buffer
                buf.seek(0) # Reset the buffer position to the beginning， so we can read
                return buf
        except Exception as e:
            if i == retries - 1:
                raise
            time.sleep(backoff ** i)

def iter_tables_from_zip(zip_bytes, zip_name_hint=""):
    """
    逐个返回 (member_name, df)。CSV/Excel 都支持。
    """
    with zipfile.ZipFile(zip_bytes) as zf: # learned from GPT that this is a good way to handle zip files，similar to "=" but do not need to close
        for member in zf.infolist(): #list all file and folder of the zip file
            name = member.filename
            lower = name.lower()
            # 忽略目录和非表格文件
            if member.is_dir():
                continue
            try:
                with zf.open(member) as f: #open the member of the zip file
                    raw = f.read()
                bio = io.BytesIO(raw) # create a buffer to read the file
                if lower.endswith(".csv"): #see if the file name ends with .csv, as we only need the csv file
                    df = pd.read_csv(bio, low_memory=False)
                elif lower.endswith(".xlsx") or lower.endswith(".xls"): #just incase if it is xls or xlsx
                    df = pd.read_excel(bio)
                else:
                    #not in our interest, so just skip
                    continue
                yield name, df
            except Exception as e:
                print(f"[WARN] 读取 {zip_name_hint}:{name} 失败 -> {e}")
                continue

#make the filename safe for the file system
def sanitize_filename(s: str) -> str:
    return re.sub(r'[^A-Za-z0-9._-]+', '_', s)

# Append all the data to one master CSV file, aligning columns by union
def append_to_master_csv(df: pd.DataFrame, csv_path: str):
    # 用列并集对齐（可能各年度列名不同）
    if not os.path.exists(csv_path):
        df.to_csv(csv_path, index=False)
        return

    # If the file already exists, read its header to align columns
    header = pd.read_csv(csv_path, nrows=0)
    all_cols = sorted(set(header.columns) | set(df.columns)) #把旧文件和新文件的列名并集
    df2 = df.reindex(columns=all_cols) #保证新数据的列顺序以及空间意义的位置一致和旧文件一致 - this is suggested by GPT during its review on my code, while not nessarily needed in my context, I think this can be added so I can use it in future context

    # CSV may also have different columns, so we need to ensure the union of columns
    if set(all_cols) != set(header.columns):
        # 需要把旧 CSV 重写为列并集（可能较慢，但稳妥）
        tmp = pd.read_csv(csv_path)
        tmp2 = tmp.reindex(columns=all_cols)
        tmp2.to_csv(csv_path, index=False, encoding="utf-8")
    
    # Now we write the new data to the master CSV
    df2.to_csv(csv_path, mode="a", index=False, header=False, encoding="utf-8")


os.makedirs(EXTRACT_DIR, exist_ok=True)
urls = list_zip_urls()
print(f"准备处理 {len(urls)} 个 ZIP…")

for i, url in enumerate(urls, 1):
    print(f"[{i}/{len(urls)}] 下载 {url}")
    zip_bytes = robust_get(url)
    zip_base = os.path.basename(urlparse(url).path)
    # By search online,
    # find, urlparse(url).path gives path of url "https://data.citybik.es/data/202301/202301-citibike-tripdata.zip?token=abc" 
                                                                                #gives "/data/202301/202301-citibike-tripdata.zip"
    # os.path.basename() gives the last part of the path, which is "202301-citibike-tripdata.zip?token=abc"
    for member_name, df in iter_tables_from_zip(zip_bytes, zip_name_hint=zip_base):
        # 1) 单表另存（方便校验）
        safe_name = sanitize_filename(f"{os.path.splitext(zip_base)[0]}__{os.path.basename(member_name)}")
        out_path = os.path.join(EXTRACT_DIR, f"{safe_name}.csv")
        df.to_csv(out_path, index=False, encoding="utf-8")
        # 2) 统一合并到一个总 CSV（按列并集对齐）
        append_to_master_csv(df, OUTPUT_CSV)
        print(f" 提取 {member_name} -> {out_path} ；已追加到 {OUTPUT_CSV}")

    print("All Done!")
    print(f"总汇总文件：{OUTPUT_CSV}")
    print(f"逐表备份目录：{EXTRACT_DIR}/")

准备处理 3 个 ZIP…
[1/3] 下载 https://s3.amazonaws.com/tripdata/202505-citibike-tripdata.zip
 提取 202505-citibike-tripdata_4.csv -> extracted/202505-citibike-tripdata__202505-citibike-tripdata_4.csv.csv ；已追加到 ../data/raw/citibike_all_trips20250820-203446.csv
 提取 202505-citibike-tripdata_5.csv -> extracted/202505-citibike-tripdata__202505-citibike-tripdata_5.csv.csv ；已追加到 ../data/raw/citibike_all_trips20250820-203446.csv
 提取 202505-citibike-tripdata_1.csv -> extracted/202505-citibike-tripdata__202505-citibike-tripdata_1.csv.csv ；已追加到 ../data/raw/citibike_all_trips20250820-203446.csv
 提取 202505-citibike-tripdata_2.csv -> extracted/202505-citibike-tripdata__202505-citibike-tripdata_2.csv.csv ；已追加到 ../data/raw/citibike_all_trips20250820-203446.csv
 提取 202505-citibike-tripdata_3.csv -> extracted/202505-citibike-tripdata__202505-citibike-tripdata_3.csv.csv ；已追加到 ../data/raw/citibike_all_trips20250820-203446.csv
All Done!
总汇总文件：../data/raw/citibike_all_trips20250820-203446.csv
逐表备份目录：extracted/
[2/3]

## Documentation
- API Source: 
    https://gbfs.citibikenyc.com/gbfs/en/station_status.json

- Scrape Source: https://s3.amazonaws.com/tripdata
- Assumptions & risks: 
    1: API rate limit: the API is currently free and public, however, this may not be the case in the future
    2: For parsing, since there are multiple zip files on the website, and our methodology uses how the zip file is named to find our data. If newer files format changed, the code cannot parse the correct file from the website
    3: the size of the data is huge, there is a hardware requirement running this program
    4: Assuming the data source only has tolarable data completeness and consistency
- Confirm `.env` is not committed.